In [ ]:
import warnings
warnings.simplefilter("ignore")
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from numpy import expand_dims
from numpy import mean
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from matplotlib import pyplot
from matplotlib.patches import Rectangle
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
import pandas as pd
sys.path.append('/home/ubuntu/Mask_RCNN/samples/coco') 
import coco
%matplotlib inline

In [57]:
# 2 classes - clown & others

class ClownDataset(Dataset):
	def load_dataset(self, dataset_dir, is_train=True):
		self.add_class("dataset", 1, "clown")
		self.add_class("dataset", 2, "others")   
		images_dir = dataset_dir + '/images/'
		annotations_dir = dataset_dir + '/annots/'
		for filename in listdir(images_dir):
			image_id = filename[:-4]
			img_path = images_dir + filename
			ann_path = annotations_dir + image_id + '.xml'
			self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path, class_ids=[0,1,2])

	def extract_boxes(self, filename):
		tree = ElementTree.parse(filename)
		root = tree.getroot()
		boxes = list()
		for box in root.findall('.//bndbox'):
			xmin = int(box.find('xmin').text)
			ymin = int(box.find('ymin').text)
			xmax = int(box.find('xmax').text)
			ymax = int(box.find('ymax').text)
			coors = [xmin, ymin, xmax, ymax]
			boxes.append(coors)
		width = int(root.find('.//size/width').text)
		height = int(root.find('.//size/height').text)
		return boxes, width, height

	def load_mask(self, image_id):
		info = self.image_info[image_id]
		path = info['annotation']
		boxes, w, h = self.extract_boxes(path)
		masks = zeros([h, w, len(boxes)], dtype='uint8')
		class_ids = list()
		for i in range(len(boxes)):
			box = boxes[i]
			row_s, row_e = box[1], box[3]
			col_s, col_e = box[0], box[2]
			if i == 0:                                    
				masks[row_s:row_e, col_s:col_e, i] = 1                   
				class_ids.append(self.class_names.index('clown'))      
			else:                                         
				masks[row_s:row_e, col_s:col_e, i] = 2                 
				class_ids.append(self.class_names.index('others'))    
		return masks, asarray(class_ids, dtype='int32')                

	def image_reference(self, image_id):
		info = self.image_info[image_id]
		return info['path']

class PredictionConfig(Config):
	NAME = "Clown_cfg"
	NUM_CLASSES = 1 + 2
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1


def evaluate_model(dataset, model, cfg):
    APs = list();
    F1_scores = list();
    PRECISION = list();
    RECALL = list();
    OVERLAPS = list();
    class_id = list(); 
    
    for image_id in dataset.image_ids:
        image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
        scaled_image = mold_image(image, cfg)
        sample = expand_dims(scaled_image, 0)
        yhat = model.detect(sample, verbose=0)
        r = yhat[0]
        AP, precision, recalls, overlaps = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'],
                                iou_threshold=0.5)
        F1_scores.append((2* (mean(precision) * mean(recalls)))/(mean(precision) + mean(recalls)))
        APs.append(AP)
        PRECISION.append(precision)
        RECALL.append(recalls)
        OVERLAPS.append(overlaps)
        class_id.append(gt_class_id)
        
    mAP = mean(APs)
    return mAP, APs, F1_scores, PRECISION, RECALL, OVERLAPS, class_id 


test_set = ClownDataset()
test_set.load_dataset('/home/ubuntu/Mask_RCNN/data/validation', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))
cfg = PredictionConfig()
model = MaskRCNN(mode = 'inference', model_dir = './', config = cfg)

model.load_weights('/home/ubuntu/Mask_RCNN/models/clown_human_ballin_100epoch/mask_rcnn_clown_cfg_0100.h5', by_name = True)
test_mAP = evaluate_model(test_set, model, cfg)

Test: 17


In [230]:
# get mAP, precision, recall, F1 (not applicable in this project)
def generateMetrics(path):
    test_set = ClownDataset()
    test_set.load_dataset(path, is_train=False)
    test_set.prepare()
    cfg = PredictionConfig()
    model = MaskRCNN(mode = 'inference', model_dir = './', config = cfg)
    model.load_weights('/home/ubuntu/Mask_RCNN/models/clown_human_ballin_100epoch/mask_rcnn_clown_cfg_0100.h5', by_name = True)
    test_mAP = evaluate_model(test_set, model, cfg)

    files = []
    for m in test_set.image_from_source_map:
        files.append(m)

    aps = []
    for ap in test_mAP[1]:
        aps.append(ap)

    f1s = []
    for f1 in test_mAP[2]:
        f1s.append(f1)

    PRECISION = []
    for p in test_mAP[3]:
        PRECISION.append(p)

    RECALL = []
    for r in test_mAP[4]:
        RECALL.append(r)

    OVERLAPS = []
    for o in test_mAP[5]:
        OVERLAPS.append(o)

    class_id = []
    for c in test_mAP[6]:
        class_id.append(c)
    
    df = pd.DataFrame({'files':files, 'AP':aps, 'F1':f1s, 'class_id':class_id,'PRECISION':PRECISION, 
                       'RECALL':RECALL, 'OVERLAPS':OVERLAPS})

    return df

In [134]:
# read how many files vs. results 

def getNumberofInstance(path, modelUse, dataUse):
    epoch1, epoch25, epoch50, epoch75, epoch100 = [], [], [], [], []
    pre_pos_files = os.listdir(path)
    for file in pre_pos_files:
        if file.endswith('_0001.h5.jpg'):
            epoch1.append(file)
        elif file.endswith('_0025.h5.jpg'):
            epoch25.append(file)
        elif file.endswith('_0050.h5.jpg'):
            epoch50.append(file)
        elif file.endswith('_0075.h5.jpg'):
            epoch75.append(file)
        elif file.endswith('_0100.h5.jpg'):
            epoch100.append(file)

    df = pd.DataFrame({modelUse:[dataUse], "epoch1":len(epoch1), "epoch25":len(epoch25), "epoch50":len(epoch50), 
                       "epoch75":len(epoch75), "epoch100":[len(epoch100)]})
    return(df)



def TrueCount(path, modelUse, dataUse):
    count = len(os.listdir(path))
    df = pd.DataFrame({modelUse:[dataUse], "TrueCount":count})
    return(df)


def main():
    PredPos_val = getNumberofInstance('/home/ubuntu/Mask_RCNN/data/validationDataBatchTest/outputMaskFrame', 
                                      "set","pred_pos_validation")
    
    PredPos_train = getNumberofInstance('/home/ubuntu/Mask_RCNN/data/trainingDataBatchTest/outputMaskFrame', 
                                      "set","pred_pos_training")
    
    PredTotal_val = getNumberofInstance('/home/ubuntu/Mask_RCNN/data/validationDataBatchTest/outputAllFrame', 
                                      "set","pred_tol_validation")
    
    PredTotal_train = getNumberofInstance('/home/ubuntu/Mask_RCNN/data/trainingDataBatchTest/outputAllFrame', 
                                      "set","pred_tol_training")
    
    prediction = pd.concat([PredPos_val, PredPos_train, PredTotal_val, PredTotal_train])
    return prediction
    
def mainTrue():
    
    TruePos_val = TrueCount('/home/ubuntu/Mask_RCNN/data/validation - True', 
                                      "set","true_pos_validation")
    TruePos_Training = TrueCount('/home/ubuntu/Mask_RCNN/data/training - True', 
                                      "set","true_pos_training")
    
    gTrue = pd.concat([TruePos_val, TruePos_Training])
    
    return gTrue
    

    
    
if __name__ == "__main__":
    
    res = main()
    resTrue = mainTrue()
    Output = res.append(resTrue).T
    Output.columns = Output.iloc[0]
    Output = Output.drop(["set"])
    

In [135]:
Output

set,pred_pos_validation,pred_pos_training,pred_tol_validation,pred_tol_training,true_pos_validation,true_pos_training
epoch1,16,64,17,65,NaN,NaN
epoch25,11,57,17,65,NaN,NaN
epoch50,10,58,17,65,NaN,NaN
epoch75,15,64,17,65,NaN,NaN
epoch100,16,64,17,65,NaN,NaN
TrueCount,NaN,NaN,NaN,NaN,7,50


In [233]:
# change the filenames of True images

def TruefilesNumber(pathtoIndex, pathTrueImage, DataUse):
    
    fileassign = generateMetrics(pathtoIndex)
    fileassign['imageID'] = fileassign.index
    IndexFiles = fileassign[['imageID', 'files']]
    IndexFiles['files'] = IndexFiles['files'].str.replace('dataset.',"")

    TrueFileConvert = []
    TrueFiles = os.listdir(pathTrueImage)
    for f in TrueFiles:
        TrueFileConvert.append(f.split(".")[0])
    Truefile = pd.DataFrame({'files': TrueFileConvert})

    f = pd.merge(IndexFiles, Truefile, on="files", how='inner')
    f[DataUse] = 'TruePositive'
    
    return f

def mainVal():
    val = TruefilesNumber('/home/ubuntu/Mask_RCNN/data/validation', '/home/ubuntu/Mask_RCNN/data/validation - True',
                    'validation')
    return val
    
def mainTrain():
    train = TruefilesNumber('/home/ubuntu/Mask_RCNN/data/training', '/home/ubuntu/Mask_RCNN/data/training - True','training')
    
    return train

if __name__ == "__main__":
    TrueVal = mainVal()
    TrueTrain = mainTrain()
    



In [319]:
def runningFiles(pathtooutput):
    ID, epoch = [], []
    files = os.listdir(pathtooutput)
    for file in files:
        ID.append(file.split('_')[0])
        epoch.append(file.split('_')[5].split('.')[0])
    inFile = pd.DataFrame({"imageID": ID, "Epoch": epoch})
    return inFile
    
    
if __name__ == "__main__":
    
    PredTrianAllfiles = runningFiles('/home/ubuntu/Mask_RCNN/data/trainingDataBatchTest/outputAllFrame')
    PredValAllfiles = runningFiles('/home/ubuntu/Mask_RCNN/data/validationDataBatchTest/outputAllFrame')
    PredTrianPosfiles = runningFiles('/home/ubuntu/Mask_RCNN/data/trainingDataBatchTest/outputMaskFrame')
    PredValPosfiles = runningFiles('/home/ubuntu/Mask_RCNN/data/validationDataBatchTest/outputMaskFrame')

In [378]:
# Validation
PredValPosfiles['validation'] = 'PredPositive'
vali = pd.merge(PredValAllfiles, PredValPosfiles, on=['imageID', 'Epoch'], how='outer')  
vali = vali.fillna('PredNegative')
listTrueVal = [str(i) for i in list(TrueVal['imageID'])]
PreVal = vali[vali['imageID'].isin(listTrueVal)]
PreVal['validation_Truth'] = 'TruePostive'
PreValf = vali[~vali['imageID'].isin(listTrueVal)]
PreValf['validation_Truth'] = 'TrueNegative'
ValidationData = pd.concat([PreVal, PreValf])

In [396]:
# Training 
PredTrianPosfiles['validation'] = 'PredPositive'
train = pd.merge(PredTrianAllfiles, PredTrianPosfiles, on=['imageID', 'Epoch'], how='outer')  
tr = train.fillna('PredNegative')
listtTrueVal = [str(i) for i in list(TrueTrain['imageID'])]
PreTrain = tr[tr['imageID'].isin(listtTrueVal)]
PreTrain['training_Truth'] = 'TruePostive'
PreTrainf = tr[~tr['imageID'].isin(listtTrueVal)]
PreTrainf['training_Truth'] = 'TrueNegative'
TrainingData = pd.concat([PreTrain, PreTrainf])

In [395]:
for e in ValidationData['Epoch'].unique():   
    ValidationData1 = ValidationData.groupby(['Epoch']).get_group(e)
    PredPos_TruePost = len(ValidationData1[(ValidationData1['validation']=='PredPositive') 
                                      &(ValidationData1['validation_Truth']=='TruePostive')])
    PredPos_TrueNeg = len(ValidationData1[(ValidationData1['validation']=='PredPositive') 
                                      &(ValidationData1['validation_Truth']=='TrueNegative')])
    PredNeg_TruePos = len(ValidationData1[(ValidationData1['validation']=='PredNegative') 
                                      &(ValidationData1['validation_Truth']=='TruePostive')])
    PredNeg_TrueNeg = len(ValidationData1[(ValidationData1['validation']=='PredNegative') 
                                      &(ValidationData1['validation_Truth']=='TrueNegative')])
    print(e, PredPos_TruePost,PredPos_TrueNeg, PredNeg_TruePos, PredNeg_TrueNeg)

0100 7 9 0 1
0050 7 3 0 7
0001 7 9 0 1
0025 7 4 0 6
0075 7 8 0 2


In [399]:
for e in TrainingData['Epoch'].unique():   
    TrainingData1 = TrainingData.groupby(['Epoch']).get_group(e)
    PredPos_TruePost = len(TrainingData1[(TrainingData1['validation']=='PredPositive') 
                                      &(TrainingData1['training_Truth']=='TruePostive')])
    PredPos_TrueNeg = len(TrainingData1[(TrainingData1['validation']=='PredPositive') 
                                      &(TrainingData1['training_Truth']=='TrueNegative')])
    PredNeg_TruePos = len(TrainingData1[(TrainingData1['validation']=='PredNegative') 
                                      &(TrainingData1['training_Truth']=='TruePostive')])
    PredNeg_TrueNeg = len(TrainingData1[(TrainingData1['validation']=='PredNegative') 
                                      &(TrainingData1['training_Truth']=='TrueNegative')])
    
    print(e, PredPos_TruePost,PredPos_TrueNeg, PredNeg_TruePos, PredNeg_TrueNeg)

0001 50 14 0 1
0050 47 11 3 4
0025 46 11 4 4
0100 50 14 0 1
0075 50 14 0 1
